<a href="https://colab.research.google.com/github/KevinTheRainmaker/Voice_Keyword_Extraction/blob/main/keyword_extraction_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install konlpy

     |████████████████████████████████| 79 kB 1.5 MB/s 
     |████████████████████████████████| 3.8 MB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 35.8 MB/s 
     |████████████████████████████████| 67 kB 2.5 MB/s 
     |████████████████████████████████| 6.5 MB 9.7 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=71980450d26fcdb31a7880a15c51f7f7bf1b385a0eafe2041156696baf2fdfc6
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 16.1 MB/s 


In [2]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [26]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [96]:
# 2022 수능 국어영역 14~17 독서 지문
doc = """
주차하거나 좁은 길을 지날 때 운전자를 돕는 장치들이 있다. 
이 중 차량 전후좌우에 장착된 카메라로 촬영한 영상을 이용하여 차량 주위 360°의 상황을 위에서 내려다본 것 같은 영상을 만들어 차 안의 모니터를 통해 운전자에게 제공하는 장치가 있다. 
운전자에게 제공되는 영상이 어떻게 만들어지는지 알아보자.
먼저 차량 주위 바닥에 바둑판 모양의 격자판을 펴 놓고 카메라로 촬영한다. 
이 장치에서 사용하는 광각 카메라는 큰 시야각을 갖고 있어 사각지대가 줄지만 빛이 렌즈를 지날 때 렌즈 고유의 곡률로 인해 영상이 중심부는 볼록하고 중심부에서 멀수록 더 휘어지는 현상, 즉 렌즈에 의한 상의 왜곡이 발생한다. 
이 왜곡에 영향을 주는 카메라 자체의 특징을 내부 변수라고 하며 왜곡 계수로 나타낸다. 이를 알 수 있다면 왜곡 모델을 설정하여 왜곡을 보정할 수 있다. 
한편 차량에 장착된 카메라의 기울어짐 등으로 인해 발생하는 왜곡의 원인을 외부 변수라고 한다.
촬영된 영상과 실세계 격자판을 비교하면 영상에서 격자판이 회전한 각도나 격자판의 위치 변화를 통해 카메라의 기울어진 각도 등을 알 수 있으므로 왜곡을 보정할 수 있다.
왜곡 보정이 끝나면 영상의 점들에 대응하는 3차원 실세계의 점들을 추정하여 이로부터 원근 효과가 제거된 영상을 얻는 시점 변환이 필요하다. 
카메라가 3차원 실세계를 2차원 영상으로 투영하면 크기가 동일한 물체라도 카메라로부터 멀리 있을수록 더 작게 나타나는데, 위에서 내려다보는 시점의 영상에서는 거리에 따른 물체의 크기 변화가 없어야 하기 때문이다.
왜곡이 보정된 영상에서의 몇 개의 점과 그에 대응하는 실세계 격자판의 점들의 위치를 알고 있다면, 영상의 모든 점들과 격자판의 점들 간의 대응 관계를 가상의 좌표계를 이용하여 기술할 수 있다. 
이 대응 관계를 이용해서 영상의 점들을 격자의 모양과 격자 간의 상대적인 크기가 실세계에서와 동일하게 유지되도록 한 평면에 놓으면 2차원 영상으로 나타난다.
이때 얻은 영상이 위에서 내려다보는 시점의 영상이 된다. 이와 같은 방법으로 구한 각 방향의 영상을 합성하면 차량 주위를 위에서 내려다본 것 같은 영상이 만들어진다.
"""

In [132]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' / '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('word_class tagging:',tokenized_doc)
print('extract nouns only:',tokenized_nouns)

word_class tagging: [('\n', 'Foreign'), ('주차', 'Noun'), ('하', 'Suffix'), ('거나', 'Noun'), ('좁은', 'Adjective'), ('길', 'Noun'), ('을', 'Josa'), ('지날', 'Verb'), ('때', 'Noun'), ('운전자', 'Noun'), ('를', 'Josa'), ('돕는', 'Verb'), ('장치', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('있다', 'Adjective'), ('.', 'Punctuation'), ('이', 'Noun'), ('중', 'Noun'), ('차량', 'Noun'), ('전후', 'Noun'), ('좌우', 'Noun'), ('에', 'Josa'), ('장착', 'Noun'), ('된', 'Verb'), ('카메라', 'Noun'), ('로', 'Josa'), ('촬영', 'Noun'), ('한', 'Josa'), ('영상', 'Noun'), ('을', 'Josa'), ('이용', 'Noun'), ('하여', 'Verb'), ('차량', 'Noun'), ('주위', 'Noun'), ('360', 'Number'), ('°', 'Foreign'), ('의', 'Noun'), ('상황', 'Noun'), ('을', 'Josa'), ('위', 'Noun'), ('에서', 'Josa'), ('내려다본', 'Verb'), ('것', 'Noun'), ('같은', 'Adjective'), ('영상', 'Noun'), ('을', 'Josa'), ('만들어', 'Verb'), ('차', 'Noun'), ('안', 'Noun'), ('의', 'Josa'), ('모니터', 'Noun'), ('를', 'Josa'), ('통해', 'Noun'), ('운전자', 'Noun'), ('에게', 'Josa'), ('제공', 'Noun'), ('하는', 'Verb'), ('장치', 'Noun'), ('가', 'Josa'), ('있

In [133]:
n_gram_range = (1,2)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('# of candidates:',len(candidates))
print('candidates:',candidates)

# of candidates: 226
candidates: ['가상' '가상 좌표계' '각도' '각도 격자' '각도 왜곡' '간의' '간의 대응' '간의 상대' '거나' '거나 운전자'
 '거리' '거리 물체' '격자' '격자 간의' '격자 모양' '격자 비교' '격자 위치' '격자 카메라' '격자 판이' '계수'
 '계수 왜곡' '고유' '고유 곡률' '곡률' '곡률 영상' '관계' '관계 가상' '관계 이용' '광각' '광각 카메라' '구한'
 '구한 방향' '기술' '기술 대응' '내부' '내부 변수' '대가' '대가 렌즈' '대응' '대응 관계' '대응 실세'
 '대응 차원' '때문' '때문 왜곡' '렌즈' '렌즈 고유' '렌즈 렌즈' '렌즈 상의' '로부터' '로부터 멀리' '먼저'
 '먼저 차량' '멀리' '멀리 시점' '모니터' '모니터 통해' '모델' '모델 설정' '모든' '모든 격자' '모양'
 '모양 격자' '물체' '물체 카메라' '물체 크기' '바닥' '바닥 바둑판' '바둑판' '바둑판 모양' '발생' '발생 왜곡'
 '방법' '방법 구한' '방향' '방향 영상' '변수' '변수 왜곡' '변수 촬영' '변화' '변화 때문' '변화 통해' '변환'
 '변환 카메라' '보정' '보정 영상' '보정 왜곡' '보정 한편' '비교' '비교 영상' '사용' '사용 광각' '상대'
 '상대 크기' '상의' '상의 왜곡' '상황' '상황 영상' '설정' '설정 왜곡' '시야' '시야 대가' '시점' '시점 변환'
 '시점 영상' '실세' '실세 격자' '실세 유지' '실세 차원' '실세 추정' '영상' '영상 거리' '영상 격자' '영상 대응'
 '영상 먼저' '영상 모니터' '영상 모든' '영상 방법' '영상 시점' '영상 실세' '영상 영상' '영상 이용' '영상 중심부'
 '영상 투영' '영상 합성' '영향' '영향 카메라' '왜곡' '왜곡 계수' '왜곡 모델' '왜곡 발생' '왜곡 보정'
 '왜곡 영향' '왜곡 원인' '외부' '외부 변수' '

In [134]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

## Cosine Similarity

In [135]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['거나 운전자', '운전자', '통해 운전자', '차원 영상', '차량 주위']


## Max Sum Similarity

In [136]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [137]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['주위 영상', '이용 영상', '합성 차량', '운전자', '차량 주위']

In [138]:
print((len(candidates)//4))
nr_can_can = (len(candidates)//4)

56


In [140]:
%time max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=nr_can_can)

CPU times: user 48.9 s, sys: 181 ms, total: 49.1 s
Wall time: 49.6 s


['고유 곡률', '기술 대응', '영상 영상', '합성 차량', '차량 주위']

## Maximal Marginal Relevance

In [149]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    word_similarity = cosine_similarity(candidate_embeddings)

    keywords_idx = [np.argmax(word_doc_similarity)]

    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [150]:
%time mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.8)

CPU times: user 6.61 ms, sys: 3 ms, total: 9.61 ms
Wall time: 9.02 ms


['차량 주위', '기술 대응', '평면 차원', '왜곡 모델', '제공 영상']